In [1]:
import agate

In [10]:
ls #looking in directory

campus-demographics.csv    school_export.csv
school demo testing.ipynb


In [3]:
# making sure CAMPUS stays text
specified_types = {
    'CAMPUS': agate.Text()
}

# import
raw = agate.Table.from_csv('campus-demographics.csv', column_types=specified_types)
print(raw) # peeking at structure

| column      | data_type |
| ----------- | --------- |
| DISTNAME    | Text      |
| CAMPNAME    | Text      |
| CNTYNAME    | Text      |
| Longitude   | Number    |
| Latitude    | Number    |
| STREET      | Text      |
| CITY        | Text      |
| ZIP         | Number    |
| CAMPUS      | Text      |
| year        | Date      |
| COUNT_ALL   | Number    |
| COUNT_BLACK | Number    |
| COUNT_HIS   | Number    |
| COUNT_WHITE | Number    |
| COUNT_AI    | Number    |
| COUNT_API   | Number    |
| COUNT_ASIAN | Number    |
| COUNT_PI    | Number    |
| COUNT_TWO   | Number    |
| COUNT_ECON  | Number    |
| COUNT_ELL   | Number    |
| PER_BLACK   | Number    |
| PER_HIS     | Number    |
| PER_WHITE   | Number    |
| PER_AI      | Number    |
| PER_API     | Number    |
| PER_ASIAN   | Number    |
| PER_PI      | Number    |
| PER_TWO     | Number    |
| PER_ECON    | Number    |
| PER_ELL     | Number    |



In [4]:
# tuple of school info columns that we will reuse in different places

school_columns = (
    'DISTNAME',
    'CAMPNAME',
    'CNTYNAME',
    'Longitude',
    'Latitude',
    'STREET',
    'CITY',
    'ZIP',
    'CAMPUS',
    'year',
    'CNT_ALL',
)

# Renameing the count columns so #chars same as PER_
# might not need to do this depending on import data
data = raw.rename(column_names = {
    'COUNT_ALL': 'CNT_ALL',
    'COUNT_BLACK': 'CNT_BLACK',
    'COUNT_HIS': 'CNT_HIS',
    'COUNT_WHITE': 'CNT_WHITE',
    'COUNT_AI': 'CNT_AI',
    'COUNT_API': 'CNT_API',
    'COUNT_ASIAN': 'CNT_ASIAN',
    'COUNT_PI': 'CNT_PI',
    'COUNT_TWO': 'CNT_TWO',
    'COUNT_ECON': 'CNT_ECON',
    'COUNT_ELL': 'CNT_ELL'
})
print(data) # print columns to see structure
data.limit(10).print_table() # peek at data

| column    | data_type |
| --------- | --------- |
| DISTNAME  | Text      |
| CAMPNAME  | Text      |
| CNTYNAME  | Text      |
| Longitude | Number    |
| Latitude  | Number    |
| STREET    | Text      |
| CITY      | Text      |
| ZIP       | Number    |
| CAMPUS    | Text      |
| year      | Date      |
| CNT_ALL   | Number    |
| CNT_BLACK | Number    |
| CNT_HIS   | Number    |
| CNT_WHITE | Number    |
| CNT_AI    | Number    |
| CNT_API   | Number    |
| CNT_ASIAN | Number    |
| CNT_PI    | Number    |
| CNT_TWO   | Number    |
| CNT_ECON  | Number    |
| CNT_ELL   | Number    |
| PER_BLACK | Number    |
| PER_HIS   | Number    |
| PER_WHITE | Number    |
| PER_AI    | Number    |
| PER_API   | Number    |
| PER_ASIAN | Number    |
| PER_PI    | Number    |
| PER_TWO   | Number    |
| PER_ECON  | Number    |
| PER_ELL   | Number    |

| DISTNAME    | CAMPNAME             | CNTYNAME | Longitude | Latitude | STREET               | ... |
| ----------- | -------------------- | 

In [5]:
# compute new columns that concatenates cnt_ and per_ columns
# should probably be a function that takes three values: category, cnt_col and pct_col
data_test = data.limit(10) # this limits our work to 10 rows for testing. Take out and fix next line for realz
data_concats = data_test.compute([
    ('BLACK_combo', agate.Formula(agate.Text(), lambda row: 'BLACK'\
                              + "|"
                              + str(row['CNT_BLACK'])\
                              + "|"
                              + str(row['PER_BLACK'])\
                             )),
    ('HIS_combo', agate.Formula(agate.Text(), lambda row: 'HISPANIC'\
                              + "|"
                              + str(row['CNT_HIS'])\
                              + "|"
                              + str(row['PER_HIS'])\
                             )),

])

### ^^^^^ will need to add the other combinations to above ^^^^^^

print(data_concats) # making sure I created it

| column      | data_type |
| ----------- | --------- |
| DISTNAME    | Text      |
| CAMPNAME    | Text      |
| CNTYNAME    | Text      |
| Longitude   | Number    |
| Latitude    | Number    |
| STREET      | Text      |
| CITY        | Text      |
| ZIP         | Number    |
| CAMPUS      | Text      |
| year        | Date      |
| CNT_ALL     | Number    |
| CNT_BLACK   | Number    |
| CNT_HIS     | Number    |
| CNT_WHITE   | Number    |
| CNT_AI      | Number    |
| CNT_API     | Number    |
| CNT_ASIAN   | Number    |
| CNT_PI      | Number    |
| CNT_TWO     | Number    |
| CNT_ECON    | Number    |
| CNT_ELL     | Number    |
| PER_BLACK   | Number    |
| PER_HIS     | Number    |
| PER_WHITE   | Number    |
| PER_AI      | Number    |
| PER_API     | Number    |
| PER_ASIAN   | Number    |
| PER_PI      | Number    |
| PER_TWO     | Number    |
| PER_ECON    | Number    |
| PER_ELL     | Number    |
| BLACK_combo | Text      |
| HIS_combo   | Text      |



In [6]:
## select just the good rows of the new table

## expand this list when all combos created
combo_columns = (
    'BLACK_combo',
    'HIS_combo'
)

# putting my schools columns list together with combo columns list
schools_combo_columns = school_columns + combo_columns

# create new table based on coluns we need
data_restructured = data_concats.select(schools_combo_columns)

print(data_restructured) #print col names to check

| column      | data_type |
| ----------- | --------- |
| DISTNAME    | Text      |
| CAMPNAME    | Text      |
| CNTYNAME    | Text      |
| Longitude   | Number    |
| Latitude    | Number    |
| STREET      | Text      |
| CITY        | Text      |
| ZIP         | Number    |
| CAMPUS      | Text      |
| year        | Date      |
| CNT_ALL     | Number    |
| BLACK_combo | Text      |
| HIS_combo   | Text      |



In [7]:
## normalize the table. Keep the school columns. Expand the combo columns

data_normalized = data_restructured.normalize(
        list(school_columns),
        list(combo_columns)
    )
print(data_normalized) # check columns
data_normalized.select(['CAMPNAME', 'property', 'value']).print_table() # peek at importnt cols

| column    | data_type |
| --------- | --------- |
| DISTNAME  | Text      |
| CAMPNAME  | Text      |
| CNTYNAME  | Text      |
| Longitude | Number    |
| Latitude  | Number    |
| STREET    | Text      |
| CITY      | Text      |
| ZIP       | Number    |
| CAMPUS    | Text      |
| year      | Date      |
| CNT_ALL   | Number    |
| property  | Text      |
| value     | Text      |

| CAMPNAME             | property    | value             |
| -------------------- | ----------- | ----------------- |
| BASTROP H S          | BLACK_combo | BLACK|218|9.6     |
| BASTROP H S          | HIS_combo   | HISPANIC|788|34.7 |
| GATEWAY SCH          | BLACK_combo | BLACK|17|31.5     |
| GATEWAY SCH          | HIS_combo   | HISPANIC|23|42.6  |
| BASTROP MIDDLE       | BLACK_combo | BLACK|93|15.5     |
| BASTROP MIDDLE       | HIS_combo   | HISPANIC|142|23.6 |
| EMILE ELEMENTARY     | BLACK_combo | BLACK|85|17       |
| EMILE ELEMENTARY     | HIS_combo   | HISPANIC|102|20.4 |
| MINA ELEMENTARY  

In [8]:
## create new columns from the split of the concatenated colunn
exploded = data_normalized.compute([
        ('CATEGORY', agate.Formula(agate.Text(), lambda row: row['value'].split("|")[0])),
        ('COUNT', agate.Formula(agate.Text(), lambda row: row['value'].split("|")[1])),
        ('PERCENT', agate.Formula(agate.Text(), lambda row: row['value'].split("|")[2])),
    ])
print(exploded)
exploded.select(['CAMPNAME', 'value', 'CATEGORY', 'COUNT', 'PERCENT']).print_table() # just to peek at it

| column    | data_type |
| --------- | --------- |
| DISTNAME  | Text      |
| CAMPNAME  | Text      |
| CNTYNAME  | Text      |
| Longitude | Number    |
| Latitude  | Number    |
| STREET    | Text      |
| CITY      | Text      |
| ZIP       | Number    |
| CAMPUS    | Text      |
| year      | Date      |
| CNT_ALL   | Number    |
| property  | Text      |
| value     | Text      |
| CATEGORY  | Text      |
| COUNT     | Text      |
| PERCENT   | Text      |

| CAMPNAME             | value             | CATEGORY | COUNT | PERCENT |
| -------------------- | ----------------- | -------- | ----- | ------- |
| BASTROP H S          | BLACK|218|9.6     | BLACK    | 218   | 9.6     |
| BASTROP H S          | HISPANIC|788|34.7 | HISPANIC | 788   | 34.7    |
| GATEWAY SCH          | BLACK|17|31.5     | BLACK    | 17    | 31.5    |
| GATEWAY SCH          | HISPANIC|23|42.6  | HISPANIC | 23    | 42.6    |
| BASTROP MIDDLE       | BLACK|93|15.5     | BLACK    | 93    | 15.5    |
| BASTROP MID

In [9]:
# reselect just the good columns
stats_columns = (
    'CATEGORY',
    'COUNT',
    'PERCENT'
)

export_columns = school_columns + stats_columns

export = exploded.select(export_columns)
print(export) # show column names
export.print_table() # peek at table
export.to_csv('school_export.csv')

| column    | data_type |
| --------- | --------- |
| DISTNAME  | Text      |
| CAMPNAME  | Text      |
| CNTYNAME  | Text      |
| Longitude | Number    |
| Latitude  | Number    |
| STREET    | Text      |
| CITY      | Text      |
| ZIP       | Number    |
| CAMPUS    | Text      |
| year      | Date      |
| CNT_ALL   | Number    |
| CATEGORY  | Text      |
| COUNT     | Text      |
| PERCENT   | Text      |

| DISTNAME    | CAMPNAME             | CNTYNAME | Longitude | Latitude | STREET               | ... |
| ----------- | -------------------- | -------- | --------- | -------- | -------------------- | --- |
| BASTROP ISD | BASTROP H S          | BASTROP  |  -95.258… |  29.439… | 1450 HTS RD          | ... |
| BASTROP ISD | BASTROP H S          | BASTROP  |  -95.258… |  29.439… | 1450 HTS RD          | ... |
| BASTROP ISD | GATEWAY SCH          | BASTROP  | -101.949… |  32.527… | 2911 COUNTY RD H     | ... |
| BASTROP ISD | GATEWAY SCH          | BASTROP  | -101.949… |  32.527… | 